## Preparation for modeling

In [76]:
import pandas as pd
import numpy as np

In [42]:
telco_raw = pd.read_csv('telco.csv',';')

In [43]:
telco_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [44]:
telco_raw.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

### Target and Features

Separate the identifier and target variable names as lists

In [45]:
custid = ['customerID']
target = ['Churn']

Separate categorical and numeric column names as lists

In [46]:
categorical = telco_raw.nunique()[telco_raw.nunique()<10].keys().tolist()
categorical.remove(target[0])

numerical = [col for col in telco_raw.columns
                if col not in custid+target+categorical]

One-hot encoding categorical variables

In [47]:
telco_raw = pd.get_dummies(data=telco_raw, columns=categorical, drop_first=True)

In [48]:
telco_raw[numerical].dtypes

tenure              int64
MonthlyCharges    float64
TotalCharges      float64
dtype: object

### Scaling

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
scaler = StandardScaler()

In [50]:
scaled_numerical = scaler.fit_transform(telco_raw[numerical])
scaled_numerical = pd.DataFrame(scaled_numerical, columns=numerical)

In [51]:
telco_raw = telco_raw.drop(columns=numerical, axis=1)
telco = telco_raw.merge(right= scaled_numerical, how = 'left', left_index=True, right_index=True)

In [52]:
telco.head()

,customerID,Churn,gender_Male,SeniorCitizen_1,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,...,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure,MonthlyCharges,TotalCharges
0,7590-VHVEG,No,0,0,1,0,0,1,0,0,...,0,0,0,1,0,1,0,-1.277445,-1.160323,-0.992611
1,5575-GNVDE,No,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0.066327,-0.259629,-0.172165
2,3668-QPYBK,Yes,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,-1.236724,-0.362660,-0.958066
3,7795-CFOCW,No,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0.514251,-0.746535,-0.193672
4,9237-HQITU,Yes,0,0,0,0,1,0,0,1,...,0,0,0,1,0,1,0,-1.236724,0.197365,-0.938874


## Modeling 

In [58]:
features = [col for col in telco.columns
                if col not in custid+target]

In [61]:
X = telco[features]
Y = telco[target]

### Supervised Learning

In [62]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [66]:
train_X, test_X, train_Y, test_Y =train_test_split(X,Y, test_size=0.25)

print(train_X.shape[0] / X.shape[0])
print(test_X.shape[0] / X.shape[0])

0.7499645037626012
0.25003549623739885


In [73]:
clf = tree.DecisionTreeClassifier(max_depth = 7, 
               criterion = 'gini', 
               splitter  = 'best')
treemodel = clf.fit(train_X, train_Y)

In [74]:
pred_Y =  treemodel.predict(test_X)

In [78]:
print("Training accuracy: ", np.round(clf.score(train_X, train_Y), 3)) 
print("Test accuracy: ", np.round(accuracy_score(test_Y, pred_Y), 3))

Training accuracy:  0.829
Test accuracy:  0.776


### Unpervised Learning

In [55]:
from sklearn.cluster import KMeans

In [80]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(telco[features])

KMeans(n_clusters=3)

In [82]:
telco[features].assign(Cluster=kmeans.labels_)

,gender_Male,SeniorCitizen_1,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,...,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure,MonthlyCharges,TotalCharges,Cluster
0,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,-1.277445,-1.160323,-0.992611,0
1,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0.066327,-0.259629,-0.172165,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,-1.236724,-0.362660,-0.958066,0
3,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0.514251,-0.746535,-0.193672,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,-1.236724,0.197365,-0.938874,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,1,0,1,0,0,0,...,1,0,1,0,0,1,-0.340876,0.665992,-0.127605,0
7039,0,0,1,1,1,0,1,1,0,0,...,1,0,1,1,0,0,1.613701,1.277533,2.242606,1
7040,0,0,1,1,0,1,0,0,0,0,...,0,0,1,0,1,0,-0.870241,-1.168632,-0.852932,0
7041,1,1,1,0,1,0,1,1,0,0,...,0,0,1,0,0,1,-1.155283,0.320338,-0.870513,0


In [ ]:
#telco[features].groupby('Cluster').mean()